Cell 1: *Introduction*:

In this notebook, we will explore the task of lip-syncing using the Wav2Lip model, which is a neural network-based approach to generating lip movements from an input audio signal. Lip-syncing is the process of synchronizing the movements of a character's lips with a spoken dialogue or sound. It is a crucial component in many applications, such as video dubbing, animation, and virtual reality, where it is essential to create a realistic and believable character. The Wav2Lip model is trained to generate lip movements that match the input audio signal, which can be used to create lip-synced videos from any input audio source. In this notebook, we will explore the implementation of the Wav2Lip model and demonstrate how it can be used to generate lip-synced videos from an input audio file.

In [ ]:
!python --version


Python 3.9.16


In [ ]:
#@title Cell 2 - Mount Google Drive
#@markdown Run this cell to mount your Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Cell 3- !pip installs and other installs
#@markdown Run this cell 1x when you run the notebook
import os
from IPython.display import clear_output

wav2lip_model_path = "/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip_gan.pth"
esrgan_model_path = "/content/drive/MyDrive/colab_projects/ESRGAN/models/RRDB_ESRGAN_x4.pth"

if not os.path.exists(wav2lip_model_path):
    !wget -O {wav2lip_model_path} "https://drive.google.com/u/0/uc?id=1dwHujXJ8YNFg4x4Slf_tF4KUQ2y80_4-&export=download"
    print("Downloading Wav2Lip model ... \n")
if not os.path.exists(esrgan_model_path):
    !wget -O {esrgan_model_path} "https://drive.google.com/u/0/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene&export=download"
    print("Downloading ESRGAN model ... \n")
clear_output()
print("\nWav2Lip & ESRGAN Setup: Done!\n")

!pip install opencv-python-headless
!pip install opencv-contrib-python-headless
!pip install face_recognition
!pip install ffmpeg-python
!pip install torch torchvision torchaudio
!pip install moviepy
!pip install tqdm
!pip install gdown
!pip install pydub
!pip install Pillow
!pip install ipywidgets

clear_output()
print("\nDone!")



Done!


In [ ]:
#@title Cell 4 - Clone & Setup Wav2Lip
#@markdown Run this cell only once to setup Wav2Lip in your Gdrive
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create folder for Colab projects
!mkdir -p '/content/drive/MyDrive/colab_projects'

# Clone Wav2Lip library to Colab projects folder
!git clone 'https://github.com/zabique/Wav2Lip' '/content/drive/MyDrive/colab_projects/Wav2Lip'

# Install dependencies and download models
%cd '/content/drive/MyDrive/colab_projects/Wav2Lip'
!pip install -r requirements.txt
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip_gan.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O '/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip.pth'
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/drive/MyDrive/colab_projects/Wav2Lip/face_detection/detection/sfd/s3fd.pth"
!pip install -U librosa==0.8.1
!pip install ffmpeg-python

# Change directory back to root
%cd '/content/'

# Done!
print("Setup complete.")

In [ ]:
#@title Cell 5 - Set all paths and upload files
# Imports and function definitions Neu 

import os
from IPython.display import display
from ipywidgets import (Accordion, Tab, Button, Label, IntProgress, FileUpload, VBox, HBox,
                        AppLayout, GridspecLayout, GridBox, Layout, HTML, IntSlider, Text, Dropdown,
                        widgets, interact, interactive)

number_of_faces = 2

path_settings = {
    'base_paths': {
        'input_folder_path': "/content/work_data/input_files/",
        'output_folder_path': "/content/work_data/output_files/",
        'temp_folder_path': "/content/work_data/temp_files/",
        'python_lib_path': "/content/drive/MyDrive/colab_projects/",
    },
    'static_paths': {
        'adjusted_audio_aac_path': "/content/work_data/temp_files/audio/adjusted_audio.aac",
        'adjusted_audio_path': "/content/work_data/temp_files/audio/adjusted_audio.wav",
        'adjusted_video_path': "/content/work_data/temp_files/face_video/adjusted_video.mp4",
        'blurdetection2_path': "/content/drive/MyDrive/colab_projects/BlurDetection2/",
        'esrgan_dejpeg_dir_path': "/content/drive/MyDrive/colab_projects/ESRGAN/dejpeg/",
        'esrgan_input_dir_path': "/content/drive/MyDrive/colab_projects/ESRGAN/input/",
        'esrgan_mask_dir_path': "/content/drive/MyDrive/colab_projects/ESRGAN/tmp/mask/",
        'esrgan_upscaled_dir_path':"/content/drive/MyDrive/colab_projects/ESRGAN/upscaled/",
        'esrgan_models_dir_path': "/content/drive/MyDrive/colab_projects/ESRGAN/models/",
        'esrgan_output_dir_path': "/content/drive/MyDrive/colab_projects/ESRGAN/output/",
        'esrgan_path': "/content/drive/MyDrive/colab_projects/ESRGAN/",
        'esrgan_tmp_dir_path': "/content/drive/MyDrive/colab_projects/ESRGAN/tmp/",
        'input_audio_path': "/content/work_data/input_files/input_audio.wav",
        'input_video_path': "/content/work_data/input_files/input_video.mp4",
        'output_background_path': "/content/work_data/temp_files/overlay_files/output_background.mp4",
        'output_face_images_folder_path': "/content/work_data/temp_files/output_face_images/",
        'silent_audio_lipsync_path': "/content/work_data/temp_files/audio/silent_audio_lipsync.wav",
        'silent_audio_path': "/content/work_data/temp_files/audio/silent_audio.wav",
        'wav2lip_path': "/content/drive/MyDrive/colab_projects/Wav2Lip/",
        'wav2lip_checkpoint_wav2lip_gan_path': "/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip_gan.pth",
        'wav2lip_checkpoint_wav2lip_path': "/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip.pth",
    },


    'dynamic_paths': {
        'face_image_path_template': "/content/work_data/temp_files/output_face_images/face_{i}.png",
        'output_face_video_path_template': "/content/work_data/temp_files/face_video/output_face_{i}.mp4",
        'input_faces_path_template': "/content/work_data/temp_files/overlay_files/output_face{i}_lipsync.mp4",
    },
     'dynamic_root_paths': {
        'face_image_root_path': "/content/work_data/temp_files/output_face_images/",
        'face_video_root_path': "/content/work_data/temp_files/face_video/",
        'overlay_files_root_path ': "/content/work_data/temp_files/overlay_files/"
    },
}

import os

def create_base_directories(path_settings):
    for key in path_settings['base_paths']:
        path = path_settings['base_paths'][key]
        if not os.path.exists(path):
            os.makedirs(path)

def create_directories_and_files(number_faces, path_settings):
    # Create base directories
    create_base_directories(path_settings)

    # Create dynamic root paths if they don't exist
    for key in path_settings['dynamic_root_paths']:
        if not os.path.exists(path_settings['dynamic_root_paths'][key]):
            os.makedirs(path_settings['dynamic_root_paths'][key])
    
    # Create dynamic paths if they don't exist
    for key in path_settings['dynamic_paths']:
        for i in range(number_faces):
            path = path_settings['dynamic_paths'][key].replace('{i}', f'{i+1}')
            directory, filename = os.path.split(path)
            if not os.path.exists(directory):
                os.makedirs(directory)
            if filename:
                open(path, 'a').close()
    
    # Create static paths if they don't exist
    for key in path_settings['static_paths']:
        path = path_settings['static_paths'][key]
        directory, filename = os.path.split(path)
        if not os.path.exists(directory):
            os.makedirs(directory)
        if filename:
            open(path, 'a').close()
    
    return path_settings

def set_default_paths(path_settings, number_of_faces):
    # Create all required directories and files
    path_settings = create_directories_and_files(number_of_faces, path_settings)

    # Set default paths and create global variables
    for main_key in path_settings:
        for subkey in path_settings[main_key]:
            variable_name = f"{subkey}"
            globals()[variable_name] = path_settings[main_key][subkey]

    return path_settings


default_settings = set_default_paths(path_settings, number_of_faces)



# Create label and progress bar widgets for the MP4 file upload
upload_mp4_label = Label(value="Select MP4 file to upload:", description="MP4 File Upload:", tooltip="Label widget for selecting the MP4 file to upload")
upload_mp4_progress_label = Label(value="Click 'UPLOAD' to select your *.mp4 file:", description="MP4 Upload Progress:", tooltip="Label widget for the MP4 file upload progress")
upload_mp4_progress_bar = IntProgress(min=0, max=100, value=0, description="input_video.mp4 upload... ", bar_style="info", orientation="horizontal", style={"description_width": "initial"}, tooltip="Progress bar widget for the MP4 file upload progress")
upload_mp4_widget = FileUpload(accept=".mp4",multiple=False,tooltip="File upload widget for the MP4 file")

# Create label and progress bar widgets for the WAV file upload
upload_wav_label = Label(value="Select WAV file to upload:",description="WAV File Upload:",tooltip="Label widget for selecting the WAV file to upload")
upload_wav_progress_label = Label(value="Click 'UPLOAD' to select your *.wav file:",description="WAV Upload Progress:",tooltip="Label widget for the WAV file upload progress")
upload_wav_progress_bar = IntProgress(min=0,max=100,value=0,description="input_audio.wav upload...",bar_style="info",orientation="horizontal",style={"description_width": "initial"},tooltip="Progress bar widget for the WAV file upload progress")
upload_wav_widget = FileUpload(accept=".wav", multiple=False, tooltip="File upload widget for the WAV file")

# Face Settings
def on_number_of_faces_slider_change(change):
    global number_of_faces
    number_of_faces = change['new']

face_settings_header = widgets.HTML(value="<h3>Face Settings:</h3>")
number_of_faces_label = widgets.Label(value="Number of Faces in the Video:")
number_of_faces_slider = widgets.IntSlider(min=1, max=10, step=1, value=number_of_faces, description="Number of Faces:")
number_of_faces_slider.observe(on_number_of_faces_slider_change, names='value')


# Select Checkpoint Folder for Wav2Lip Models and get the selected model path
wav2lip_checkpoints_folder_path_label = widgets.Label(value="Select Checkpoint Folder for Wav2Lip Models:")
wav2lip_checkpoint_wav2lip_path = "/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip.pth"
wav2lip_checkpoint_wav2lip_gan_path = "/content/drive/MyDrive/colab_projects/Wav2Lip/checkpoints/wav2lip_gan.pth"
wav2lip_checkpoints_folder_path_options = {
    "wav2lip.pth": wav2lip_checkpoint_wav2lip_path,
    "wav2lip_gan.pth": wav2lip_checkpoint_wav2lip_gan_path
}
wav2lip_checkpoints_folder_path_dropdown = widgets.Dropdown(options=wav2lip_checkpoints_folder_path_options.keys(), value=list(wav2lip_checkpoints_folder_path_options.keys())[0], description="Wav2Lip Checkpoints Folder:")

def get_selected_model_path():
    return wav2lip_checkpoints_folder_path_options[wav2lip_checkpoints_folder_path_dropdown.value]

# Load Path to Wav2Lip model
wav2lip_selected_checkpoint_path = get_selected_model_path()


# Main Folder Settings
main_folder_settings_header = widgets.HTML(value="<h3>Main Folder Settings:</h3>")
input_folder_path_label = widgets.Label(value="Input Folder:", tooltip="Path to the input folder")
input_folder_path_text = widgets.Text(value=input_folder_path,placeholder=input_folder_path,tooltip="Path to the input folder")
output_folder_path_label = widgets.Label(value="Output Folder:", tooltip="Path to the output folder")
output_folder_path_text = widgets.Text(value=output_folder_path,placeholder=output_folder_path, tooltip="Path to the output folder")
temp_folder_path_label = widgets.Label(value="Temp Folder:", tooltip="Path to the temp folder")
temp_folder_path_text = widgets.Text(value=temp_folder_path,placeholder=temp_folder_path, tooltip="Path to the temp folder")

# Video/Audio Path Settings
video_audio_settings_header = widgets.HTML(value="<h3>Video/Audio Path Settings:</h3>")
input_video_path_label = widgets.Label(value="Input Video Path:", tooltip="Path to the input video")
input_video_path_text = widgets.Text(value=input_video_path,placeholder=input_video_path, tooltip="Path to the input video")
input_audio_path_label = widgets.Label(value="Input Audio Path:", tooltip="Path to the input audio")
input_audio_path_text = widgets.Text(value=input_audio_path,placeholder=input_audio_path, tooltip="Path to the input audio")
adjusted_video_path_label = widgets.Label(value="Adjusted Video Path:", tooltip="Path to the adjusted video")
adjusted_video_path_text = widgets.Text(value=adjusted_video_path,placeholder=adjusted_video_path, tooltip="Path to the adjusted video")
adjusted_audio_path_label = widgets.Label(value="Adjusted Audio Path:", tooltip="Path to the adjusted audio")
adjusted_audio_path_text = widgets.Text(value=adjusted_audio_path,placeholder=adjusted_audio_path, tooltip="Path to the adjusted audio")
adjusted_audio_aac_path_label = widgets.Label(value="Adjusted Audio AAC Path:", tooltip="Path to the adjusted audio AAC")
adjusted_audio_aac_path_text = widgets.Text(value=adjusted_audio_aac_path,placeholder=adjusted_audio_aac_path, tooltip="Path to the adjusted audio AAC")
silent_audio_path_label = widgets.Label(value="Silent Audio Path:", tooltip="Path to the silent audio")
silent_audio_path_text = widgets.Text(value=silent_audio_path,placeholder=adjusted_audio_aac_path, tooltip="Path to the silent audio AAC")
silent_audio_lipsync_path_label = widgets.Label(value="Silent Audio Lipsync Path:", tooltip="Path to the silent audio lipsync")
silent_audio_lipsync_path_text = widgets.Text(value=silent_audio_lipsync_path,placeholder=silent_audio_lipsync_path, tooltip="Path to the silent audio lipsync")
output_background_path_label = widgets.Label(value="Output Background Path:", tooltip="Path to the output background")
output_background_path_text = widgets.Text(value=output_background_path,placeholder=output_background_path, tooltip="Path to the output background")

# GDrive ESRGAN directories
gdrive_esrgan_settings_header = widgets.HTML(value="<h3>GDrive ESRGAN directories:</h3>")
esrgan_path_label = widgets.Label(value="ESRGAN Path:", tooltip="Path to the ESRGAN")
esrgan_path_text = widgets.Text(value=esrgan_path,placeholder=esrgan_path, tooltip="Path to the ESRGAN")
esrgan_tmp_dir_path_label = widgets.Label(value="ESRGAN Temp Directory:", tooltip="Path to ESRGAN the temporary directory")
esrgan_tmp_dir_path_text = widgets.Text(value=esrgan_tmp_dir_path,placeholder=esrgan_tmp_dir_path, tooltip="Path to the ESRGAN temporary directory")
esrgan_mask_dir_path_label = widgets.Label(value="ESRGAN Mask Directory:", tooltip="Path to the ESRGAN mask directory")
esrgan_mask_dir_path_text = widgets.Text(value=esrgan_mask_dir_path,placeholder=esrgan_mask_dir_path, tooltip="Path to the ESRGAN mask directory")
esrgan_input_dir_path_label = widgets.Label(value="ESRGAN Input Directory:", tooltip="Path to the ESRGAN input directory")
esrgan_input_dir_path_text = widgets.Text(value=esrgan_input_dir_path,placeholder=esrgan_input_dir_path,tooltip="Path to the ESRGAN input directory")
esrgan_dejpeg_dir_path_label = widgets.Label(value="ESRGAN Dejpeg Directory:", tooltip="Path to the ESRGAN dejpeg directory")
esrgan_dejpeg_dir_path_text = widgets.Text(value=esrgan_input_dir_path,placeholder=esrgan_input_dir_path,tooltip="Path to the ESRGAN dejpeg directory")
esrgan_upscaled_dir_path_label = widgets.Label(value="ESRGAN Upscaled Directory:", tooltip="Path to the ESRGAN upscaled_directory")
esrgan_upscaled_dir_path_text = widgets.Text(value=esrgan_upscaled_dir_path,placeholder=esrgan_upscaled_dir_path, tooltip="Path to the ESRGAN upscaled directory")
esrgan_output_dir_path_label = widgets.Label(value="ESRGAN Output Directory:", tooltip="Path to the ESRGAN output directory")
esrgan_output_dir_path_text = widgets.Text(value=esrgan_output_dir_path,placeholder=esrgan_output_dir_path, tooltip="Path to the output ESRGAN directory")
esrgan_models_dir_path_label = widgets.Label(value="ESRGAN Models Directory:", tooltip="Path to the ESRGAN models directory")
esrgan_models_dir_path_text = widgets.Text(value=esrgan_models_dir_path,placeholder=esrgan_models_dir_path, tooltip="Path to the ESRGAN models directory")

# GDrive BlurDedection2 directories
gdrive_blurdetection_settings_header = widgets.HTML(value="<h3>GDrive BlurDedection2 directories:</h3>")
blurdetection2_path_label = widgets.Label(value="BlurDedection2 Path:", tooltip="Path to the BlurDedection2")
blurdetection2_path_text = widgets.Text(value=blurdetection2_path,placeholder=blurdetection2_path, tooltip="Path to the BlurDedection2")




# Define the function that will be called when the button is clicked
def on_upload_button_click(path_settings,number_of_faces):
    # Create the directories for input files if they don't exist
    if not os.path.exists(input_folder_path):
        create_directories_and_files(path_settings, number_of_faces)

    # Upload the MP4 file
    for filename, filedata in upload_mp4_widget.value.items():
        with open(input_video_path, "wb") as f:
            f.write(filedata["content"])

        total_size = len(filedata["content"])
        uploaded_size = total_size
        upload_mp4_progress_bar.value = int(uploaded_size / total_size * 100)

    # Update the value of the MP4 progress bar widget to indicate that the MP4 file has been uploaded
    upload_mp4_progress_bar.value = 100

    # Upload the WAV file
    for filename, filedata in upload_wav_widget.value.items():
        with open(input_audio_path, "wb") as f:
            f.write(filedata["content"])

        total_size = len(filedata["content"])
        uploaded_size = total_size
        upload_wav_progress_bar.value = int(uploaded_size / total_size * 100)

    # Update the value of the WAV progress bar widget to indicate that the WAV file has been uploaded
    upload_wav_progress_bar.value = 100


def reset_to_default_values(button):
    set_default_paths(path_settings, number_of_faces)



# Connect all Buttons
save_upload_files_button = widgets.Button(description="Upload Files",tooltip="Click to start uploading your MP4 and WAV files")
save_upload_files_button.on_click(lambda _: on_upload_button_click(path_settings, number_of_faces))



reset_to_default_values_button = widgets.Button(description="Reset to default values", tooltip="Reset all paths to default values.")
reset_to_default_values_button.on_click(reset_to_default_values)



# Layout settings
global_layout_settings = {
    "width": "500px",
    "height": "50px",
    "border_color": "#d8dbe0",
    "border_width": "1px",
    "border_style": "solid",
    "background_color": "#f5f5f7",
    "color": "#1d1d1f",
    "font_family": "Apple SD Gothic Neo",
    "font_size": "16px",
    "font_weight": "normal",
    "button_color": "#1e90ff",
    "heading_size": "16px",
    "heading_color": "#1d1d1f",
    "heading_font_family": "Apple SD Gothic Neo",
    "heading_font_weight": "bold"
}

light_theme_settings = {
    "width": "500px",
    "height": "50px",
    "border_color": "#d8dbe0",
    "border_width": "1px",
    "border_style": "solid",
    "background_color": "#f5f5f7",
    "color": "#1d1d1f",
    "font_family": "Arial, sans-serif",
    "font_size": "16px",
    "font_weight": "normal",
    "button_color": "#1e90ff",
    "heading_size": "16px",
    "heading_color": "#1d1d1f",
    "heading_font_family": "Arial, sans-serif",
    "heading_font_weight": "bold"
}

dark_theme_settings = {
    "width": "500px",
    "height": "50px",
    "border_color": "#d8dbe0",
    "border_width": "1px",
    "border_style": "solid",
    "background_color": "#1d1d1f",
    "color": "#f5f5f7",
    "font_family": "Arial, sans-serif",
    "font_size": "16px",
    "font_weight": "normal",
    "button_color": "#1e90ff",
    "heading_size": "16px",
    "heading_color": "#f5f5f7",
    "heading_font_family": "Arial, sans-serif",
    "heading_font_weight": "bold"
}

dark_theme_settings2 = {
    "width": "500px",
    "height": "50px",
    "border_color": "#ff5733",
    "border_width": "2px",
    "border_style": "solid",
    "background_color": "#262626",
    "color": "#fff",
    "font_family": "Arial, sans-serif",
    "font_size": "16px",
    "font_weight": "normal",
    "button_color": "#1e90ff",
    "heading_size": "16px",
    "heading_color": "#fff",
    "heading_font_family": "Arial, sans-serif",
    "heading_font_weight": "bold"
}


def layout_settings(widget, width=None, height=None, border_color=None, background_color=None, border_style=None, color_theme=None, font_family=None, font_size=None, font_weight=None, heading_size=None, heading_color=None, heading_font_family=None, heading_font_weight=None):
    try:
        widget_type = widget.__class__.name
    except AttributeError:
        widget_type = getattr(widget.__class__, "__name__", None)
    widget.layout.padding = "5px"

    if color_theme is None:
        color_theme = global_layout_settings

    if border_color is None:
        border_color = color_theme['border_color']
    if background_color is None:
        background_color = color_theme['background_color']
    if width is None:
        width = color_theme['width']
    if height is None:
        height = color_theme['height']
    if font_family is None:
        font_family = color_theme['font_family']
    if font_size is None:
        font_size = color_theme['font_size']
    if font_weight is None:
        font_weight = color_theme['font_weight']
    if heading_size is None:
        heading_size = color_theme['heading_size']
    if heading_color is None:
        heading_color = color_theme['heading_color']
    if heading_font_family is None:
        heading_font_family = color_theme['heading_font_family']
    if heading_font_weight is None:
        heading_font_weight = color_theme['heading_font_weight']

    if widget_type == 'Slider':
        widget.layout.border_radius = "5px"
        widget.style.width = width
        widget.style.height = height
        widget.style.handle_color = color_theme['handle_color']
        widget.style.slider_color = color_theme['slider_color']
        widget.style.progress_color = color_theme['progress_color']
    elif widget_type == 'Text':
        widget.style.description_width = "initial"
        widget.style.font_family = font_family
        widget.style.font_size = font_size
        widget.style.font_weight = font_weight
        widget.layout.background_color = background_color
        widget.layout.width = width
        widget.layout.height = height
        if widget.placeholder:
            widget.placeholder_style = {'color': '#a4a4a4'}
        if widget.description:
            widget.description_style = {
                'color': color_theme['color'],
                'font-size': heading_size,
                'font-family': heading_font_family,
                'font-weight': heading_font_weight
            }
    elif widget_type == 'Label':
        widget.style.font_family = font_family
        widget.style.font_size = font_size
        widget.style.font_weight = font_weight
        widget.layout.background_color = background_color
        widget.layout.width = width
        widget.layout.height = height
        widget.style.color = heading_color
        widget.style.font_size = heading_size
        widget.style.font_family = heading_font_family
        widget.style.font_weight = heading_font_weight
    elif widget_type == 'VBox' or widget_type == 'HBox':
        widget.layout.background_color = background_color
        widget.layout.width = width
        widget.layout.height = height
    elif widget_type == 'FileUpload':
        widget.style.button_color = color_theme['button_color']
        widget.layout.width = width
        widget.layout.height = height
        if widget.description:
            widget.description_style = {
                'color': color_theme['color'],
                'font-size': heading_size,
                'font-family': heading_font_family,
                'font-weight': heading_font_weight
            }
    elif widget_type == 'Accordion':
        for child in widget.children:
            child.header_style = {
                'background-color': background_color,
                'color': heading_color,
                'font-size': heading_size,
                'font-family': heading_font_family,
                'font-weight': heading_font_weight
            }
            child.layout.width = width
            child.layout.height = height
            layout_settings(child, color_theme=color_theme, heading_size=heading_size, heading_color=heading_color, heading_font_family=heading_font_family, heading_font_weight=heading_font_weight)
    elif widget_type == 'Tab':
        for child in widget.children:
            child.layout.width = width
            child.layout.height = height
            layout_settings(child, color_theme=color_theme, heading_size=heading_size, heading_color=heading_color, heading_font_family=heading_font_family, heading_font_weight=heading_font_weight)
            if child.title:
                child.title_style = {
                    'color': heading_color,
                    'font-size': heading_size,
                    'font-family': heading_font_family,
                    'font-weight': heading_font_weight
                }
    else:
        widget.style.font_family = font_family
        widget.style.font_size = font_size
        widget.style.font_weight = font_weight
        widget.style.color = heading_color
        widget.style.font_size = heading_size
        widget.style.font_family = heading_font_family
        widget.style.font_weight = heading_font_weight
    return widget


general_settings_accordion = Accordion(children=[
    HBox([
        VBox([
            layout_settings(face_settings_header),
            layout_settings(number_of_faces_label),
            layout_settings(number_of_faces_slider),
            layout_settings(upload_mp4_label),
            layout_settings(upload_mp4_widget),  
 
            layout_settings(upload_mp4_progress_bar), 
            layout_settings(upload_wav_label),
            layout_settings(upload_wav_widget), 
 
            layout_settings(upload_wav_progress_bar), 
              
        ]),
        VBox([
            wav2lip_checkpoints_folder_path_label,
            wav2lip_checkpoints_folder_path_dropdown,
            widgets.Label("Set all paths / Upload Files *required to run the app"),
            layout_settings(save_upload_files_button),
            widgets.Label("Reset to Default Values (Optional)"),
            layout_settings(reset_to_default_values_button),
            
        ], layout=widgets.Layout(margin='0 0 0 20px'))
    ]),
])
general_settings_accordion.set_title(0, "General Settings")


path_settings_accordion = Accordion(children=[
    VBox([
        layout_settings(input_folder_path_label),
        layout_settings(input_folder_path_text),
        layout_settings(output_folder_path_label),
        layout_settings(output_folder_path_text),
        layout_settings(temp_folder_path_label),
        layout_settings(temp_folder_path_text),
    ]),
])
path_settings_accordion.set_title(0, "Path Settings")

video_audio_settings_accordion = Accordion(children=[
    VBox([
        layout_settings(adjusted_video_path_label),
        layout_settings(adjusted_video_path_text),
        layout_settings(adjusted_audio_path_label),
        layout_settings(adjusted_audio_path_text),
        layout_settings(adjusted_audio_aac_path_label),
        layout_settings(adjusted_audio_aac_path_text),
        layout_settings(silent_audio_path_label),
        layout_settings(silent_audio_path_text),
        layout_settings(silent_audio_lipsync_path_label),
        layout_settings(silent_audio_lipsync_path_text),
        layout_settings(output_background_path_label),
        layout_settings(output_background_path_text),
    ]),
])
video_audio_settings_accordion.set_title(0, "Video and Audio Settings")
esrgan_settings_accordion = Accordion(children=[
    VBox([
        layout_settings(esrgan_path_label),
        layout_settings(esrgan_path_text),
        layout_settings(esrgan_tmp_dir_path_label),
        layout_settings(esrgan_tmp_dir_path_text),
        layout_settings(esrgan_mask_dir_path_label),
        layout_settings(esrgan_mask_dir_path_text),
        layout_settings(esrgan_input_dir_path_label),
        layout_settings(esrgan_input_dir_path_text),
        layout_settings(esrgan_dejpeg_dir_path_label),
        layout_settings(esrgan_dejpeg_dir_path_text),
        layout_settings(esrgan_upscaled_dir_path_label),
        layout_settings(esrgan_upscaled_dir_path_text),
        layout_settings(esrgan_output_dir_path_label),
        layout_settings(esrgan_output_dir_path_text),
        layout_settings(esrgan_models_dir_path_label),
        layout_settings(esrgan_models_dir_path_text),
    ]),
])
esrgan_settings_accordion.set_title(0, "ESRGAN Settings")

blurdetection_settings_accordion = Accordion(children=[
    VBox([
        layout_settings(blurdetection2_path_label),
        layout_settings(blurdetection2_path_text),
    ]),
])
blurdetection_settings_accordion.set_title(0, "Blur Detection Settings")

# Group accordions into tabs
settings_tab = Tab(children=[
    general_settings_accordion,
    path_settings_accordion,
    video_audio_settings_accordion,
    esrgan_settings_accordion,
    blurdetection_settings_accordion,
])
settings_tab.layout = widgets.Layout(flex='1')

settings_tab.set_title(0, "General")
settings_tab.set_title(1, "Paths")
settings_tab.set_title(2, "Video & Audio")
settings_tab.set_title(3, "ESRGAN")
settings_tab.set_title(4, "Blur Detection")

grid_box = GridBox([
    settings_tab,
], layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='center', justify_content='center', width='100%'))
display(grid_box)



GridBox(children=(Tab(children=(Accordion(children=(HBox(children=(VBox(children=(HTML(value='<h3>Face Setting…

In [ ]:
#@title Cell 6 Setup für ESRG Supermodel 
#@markdown This cell needs to be run only once.

import os
force_setup = False

pip_packages = 'typer rich gmic'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
#if not os.path.isdir('/content/drive') and force_setup == False:
#  from google.colab import drive
#  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

%cd /content/drive/MyDrive/colab_projects
!git clone https://github.com/olaviinha/ESRGAN.git
!git clone https://github.com/olaviinha/BlurDetection2.git
%cd "/content/drive/MyDrive/colab_projects/ESRGAN"

dir_tmp = '/content/drive/MyDrive/colab_projects/ESRGAN/tmp/'
dir_mask = '/content/drive/MyDrive/colab_projects/ESRGAN/tmp/mask/'
dir_input = '/content/drive/MyDrive/colab_projects/ESRGAN/input/'
dir_dejpeg = '/content/drive/MyDrive/colab_projects/ESRGAN/dejpeg/'
dir_upscaled = '/content/drive/MyDrive/colab_projects/ESRGAN/upscaled/'
dir_output = '/content/drive/MyDrive/colab_projects/ESRGAN/output/'
dir_models = '/content/drive/MyDrive/colab_projects/ESRGAN/models/'
create_dirs([dir_tmp, dir_input, dir_mask, dir_dejpeg, dir_upscaled, dir_output, dir_models])

#----------------------------------------

import requests, gmic, cv2
from google.colab import output
from google.colab.patches import cv2_imshow
from IPython.display import Image
from datetime import datetime
force_setup = False

downloaded_models = []
#----------------------------------------

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

Setup finished.


In [ ]:
#@title Cell 8 - Video and Audio Adjustment
#@markdown Run this cell to match the duration of your .mp4 and .wav files
import os
import cv2
import numpy as np
import moviepy.editor as mp
import shutil
from moviepy.video.fx.resize import resize
from moviepy.audio.AudioClip import AudioArrayClip
import subprocess


# Check if input_video.mp4 and input_audio.wav exist
if not (os.path.exists(input_video_path) and os.path.exists(input_audio_path)):
    raise Exception("input_video.mp4 and/or input_audio.wav not found in the input folder.")
else:
    print("Input video and audio files found.")

def create_silent_audio(duration, filepath, sample_rate=44100):
    audio_array = np.zeros((int(np.ceil(float(duration)) * sample_rate), 2), dtype=np.float32)
    silent_audio = AudioArrayClip(audio_array, fps=sample_rate)
    silent_audio.write_audiofile(filepath)


def get_audio_codec(input_audio):
    command = f'ffprobe -v error -show_entries stream=codec_name -of default=noprint_wrappers=1:nokey=1 -select_streams a:0 "{input_audio}"'
    codec = subprocess.check_output(command, shell=True).decode('utf-8').strip()
    return codec

def get_video_codec(input_video):
    command = f'ffprobe -v error -show_entries stream=codec_name -of default=noprint_wrappers=1:nokey=1 -select_streams v:0 "{input_video}"'
    codec = subprocess.check_output(command, shell=True).decode('utf-8').strip()
    return codec

def convert_audio_to_aac(input_audio, output_audio):
    command = f'ffmpeg -i "{input_audio}" -c:a aac "{output_audio}"'
    subprocess.call(command, shell=True)

def adjust_video_audio(input_video, input_audio, adjusted_video, adjusted_audio, silent_audio_path, adjusted_audio_aac, silent_audio_lipsync_path):
    # Load input video and audio
    video = mp.VideoFileClip(input_video_path)
    audio = mp.AudioFileClip(input_audio_path)

    # Get durations
    video_duration = video.duration
    audio_duration = audio.duration

    if video_duration > audio_duration:
        print(f"\nVideo duration is longer than the audio. I will add silence {video_duration - audio_duration:.2f} seconds at the end of your audio.\n")
        shutil.copy(input_video_path, adjusted_video)
        silent_audio_duration = video_duration - audio_duration
        create_silent_audio(silent_audio_duration, silent_audio_path)
        silent_audio = mp.AudioFileClip(silent_audio_path)
        audio_adjusted = mp.concatenate_audioclips([audio, silent_audio])
        audio_adjusted.write_audiofile(adjusted_audio)

    elif video_duration < audio_duration:
        print(f"\nVideo duration is shorter than the audio. I will adjust the speed of the video, by adding {audio_duration - video_duration:.2f} seconds to the duration of your video!\n")
        shutil.copy(input_audio, adjusted_audio)
        speed_factor = audio_duration / video_duration
        video_adjusted = video.fx(mp.vfx.speedx, speed_factor)
        video_adjusted.write_videofile(adjusted_video, audio=None)

    else:
        print("\nVideo duration = audio duration. I will just move the files to the right directory for further processing!\n")
        shutil.copy(input_video, adjusted_video)
        shutil.copy(input_audio, adjusted_audio)

    # Convert audio to aac
    convert_audio_to_aac(adjusted_audio, adjusted_audio_aac)

    # Create silent_audio_lipsync.wav with the same duration as adjusted_audio
    adjusted_audio_duration = mp.AudioFileClip(adjusted_audio).duration
    create_silent_audio(adjusted_audio_duration, silent_audio_lipsync_path)

# Call adjust_video_audio function
# Call adjust_video_audio function
adjust_video_audio(
    input_video_path,
    input_audio_path,
    adjusted_video_path,
    adjusted_audio_path,
    silent_audio_path,
    adjusted_audio_aac_path,
    silent_audio_lipsync_path
)


Input video and audio files found.

Video duration is longer than the audio. I will add silence 0.02 seconds at the end of your audio.

MoviePy - Writing audio in /content/work_data/temp_files/audio/silent_audio.wav


MoviePy - Done.
MoviePy - Writing audio in /content/work_data/temp_files/audio/adjusted_audio.wav


MoviePy - Done.
MoviePy - Writing audio in /content/work_data/temp_files/audio/silent_audio_lipsync.wav


MoviePy - Done.


In [ ]:
#@title Cell 9 Create a video for each face
#@markdown Run this cell to track and seperate all faces in the video
import os
import cv2
import numpy as np
import face_recognition
from moviepy.editor import ImageClip
from tqdm import tqdm

def create_face_mask(frame, face_location, frame_number, face_index):
    top, right, bottom, left = face_location
    height, width, _ = frame.shape

    padding_width = int((right - left) * 0.8)
    padding_height = int((bottom - top) * 0.8)

    left = max(0, left - padding_width)
    top = max(0, top - padding_height)
    right = min(width, right + padding_width)
    bottom = min(height, bottom + padding_height)

    face_mask = np.zeros((height, width), dtype=np.uint8)
    face_mask[top:bottom, left:right] = 255

    return face_mask

def interpolate_face_position(face_locations):
    x_coords = [loc[1] + (loc[3] - loc[1]) // 2 for loc in face_locations]
    y_coords = [loc[0] + (loc[2] - loc[0]) // 2 for loc in face_locations]
    new_x = int(sum(x_coords) / len(x_coords))
    new_y = int(sum(y_coords) / len(y_coords))
    return new_x, new_y

def save_face_image(input_frame, face_location, face_index, output_folder):
    top, right, bottom, left = face_location
    face_image = input_frame[top:bottom, left:right]
    face_image = cv2.resize(face_image, (400, 255))
    cv2.imwrite(os.path.join(output_folder, f"face_{face_index + 1}.png"), face_image)

def separate_faces_and_background(input_video, output_background_path, face_image_path_template, face_image_root_path):

    video_capture = cv2.VideoCapture(input_video)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    success, frame = video_capture.read()
    if not success:
        raise Exception("Could not read the input video.")

    # Define face_locations after reading the first frame from the video
    face_locations = face_recognition.face_locations(frame)

    background_writer = cv2.VideoWriter(output_background_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))
    # Create a video writer for each face
    face_writers = [cv2.VideoWriter(f"{output_face_video_path_template.format(i=i + 1)}", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height)) for i in range(len(face_locations))]
    
    
    if not success:
        raise Exception("Could not read the input video.")

    frame_number = 0
    face_locations = face_recognition.face_locations(frame)
    face_trackers = {i: cv2.TrackerKCF_create() for i in range(len(face_locations))}

    for i, face_location in enumerate(face_locations):
        top, right, bottom, left = face_location
        tracker = face_trackers[i]
        tracker.init(frame, (left, top, right - left, bottom - top))

        # Save the first frame of each face as an image
        save_face_image(frame, face_location, i, output_face_images_folder_path)

    face_positions_list = []
    face_masks_list = []

    with tqdm(total=total_frames, desc="Processing frames") as progress_bar:
        while success:
            success, frame = video_capture.read()
            if not success:
                break

            frame_number += 1

            updated_face_locations = []

            for i, tracker in face_trackers.items():
                success, bbox = tracker.update(frame)
                if success:
                    x, y, w, h = [int(v) for v in bbox]
                    updated_face_locations.append((y, x + w, y + h, x))
                else:
                    x, y = interpolate_face_position(face_locations[i])
                    updated_face_locations.append((y - h // 2, x + w // 2, y + h // 2, x - w // 2))

            face_positions_list.append(updated_face_locations)

            background_frame = frame.copy()

            current_frame_face_masks = []
            for i, face_location in enumerate(updated_face_locations):
                face_mask = create_face_mask(frame, face_location, frame_number, i)
                current_frame_face_masks.append(face_mask)

                face_frame = frame.copy()
                face_frame[np.where(face_mask == 0)] = 0
                background_frame[np.where(face_mask != 0)] = 0
                face_writers[i].write(face_frame)

            face_masks_list.append(current_frame_face_masks)
            background_writer.write(background_frame)
            progress_bar.update(1)

    video_capture.release()
    background_writer.release()
    for face_writer in face_writers:
        face_writer.release()

    print("Finished processing input video.")
    return face_positions_list, face_masks_list

face_positions_list, face_masks_list = separate_faces_and_background(adjusted_video_path, output_background_path, output_face_video_path_template, face_image_path_template)

print("Separated faces from background - Hooray!!")


Processing frames: 100%|█████████▉| 438/439 [01:25<00:00,  5.11it/s]

Finished processing input video.
Separated faces from background - Hooray!!


In [ ]:
# VERSION 3.0

%gui qt  
import os
import shutil
from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QGroupBox,
                             QFormLayout, QLabel, QLineEdit, QPushButton,
                             QSpinBox, QHBoxLayout, QToolBox, QFileDialog,
                             QSizePolicy, QMessageBox)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont, QPalette
from functools import partial

theme_dict = {
    "QGroupBox": {
        "background-color": "#F0F0F0",
        "color": "#000000",
        "font": "Arial",
        "font-weight": "normal",
        "padding": "5px",
        "border": "1px solid #000",
        "border-radius": "5px",
        "margin": "5px",
        "alignment": Qt.AlignCenter,
    },
    "QLabel": {
        "color": "#000000",
        "font": "Arial",
        "font-weight": "normal",
        "alignment": Qt.AlignLeft,
        "margin": "2px",
    },
    "QLineEdit": {
        "background-color": "#FFFFFF",
        "color": "#000000",
        "font": "Arial",
        "font-weight": "normal",
        "padding": "2px",
        "border": "1px solid #000",
        "border-radius": "2px",
        "margin": "2px",
    },
    "QSpinBox": {
        "background-color": "#FFFFFF",
        "color": "#000000",
        "font": "Arial",
        "font-weight": "normal",
        "padding": "2px",
        "border": "1px solid #000",
        "border-radius": "2px",
        "margin": "2px",
    },
    "QPushButton": {
        "background-color": "#E0E0E0",
        "color": "#000000",
        "font": "Arial",
        "font-weight": "normal",
        "padding": "5px",
        "border": "1px solid #000",
        "border-radius": "3px",
        "margin": "5px",
    },
    "QMainWindow": {
        "background-color": "#fff",
        "font": "Arial",
        "font-weight": "normal",
        "padding": "10px"
    } 
}

def apply_widget_styles(widget, theme_dict):
    if isinstance(widget, QWidget):
        widget_type = widget.__class__.__name__
        if widget_type in theme_dict:
            style = theme_dict[widget_type]
            if 'background-color' in style or 'color' in style:
                bg_color = style.get('background-color', '#000')
                fg_color = style.get('color', '#fff')
                widget.setStyleSheet(f'background-color: {bg_color}; color: {fg_color};')

            if 'font' in style or 'font-weight' in style:
                font = QFont()
                font.setFamily(style.get('font', 'Arial'))
                font.setWeight(QFont.Normal if style.get('font-weight', 'normal') == 'normal' else QFont.Bold)
                widget.setFont(font)

            if 'border' in style:
                widget.setStyleSheet(widget.styleSheet() + f'border: {style["border"]};')
            if 'border-radius' in style:
                widget.setStyleSheet(widget.styleSheet() + f'border-radius: {style["border-radius"]};')

            if 'padding' in style:
                widget.setContentsMargins(
                    *tuple(map(int, style['padding'].split('px')) * 4)
                )

            if 'margin' in style:
                widget.layout().setContentsMargins(
                    *tuple(map(int, style['margin'].split('px')) * 4)
                )

            if 'alignment' in style:
                widget.setAlignment(style['alignment'])

        for child in widget.children():
            apply_widget_styles(child, theme_dict)

class Wav2LipApp:
    def __init__(self):
        self.number_of_faces = 2
        self.path_settings = {
        'base_paths': {
                'input_folder_path': "C:\Users\pwalc\Documents\python_work_data\input_files\",
                'output_folder_path': "C:\Users\pwalc\Documents\python_work_data\output_files\",
                'temp_folder_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\",
                'python_lib_path': "C:\PythonLibraries\",
            },
            'static_paths': {
                'adjusted_audio_aac_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\audio\adjusted_audio.aac",
                'adjusted_audio_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\audio\adjusted_audio.wav",
                'adjusted_video_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\face_video\adjusted_video.mp4",
                'blurdetection2_path': "C:\PythonLibraries\BlurDetection2\",
                'esrgan_dejpeg_dir_path': "C:\Users\pwalc\Documents\python_work_data\ESRGAN\dejpeg\",
                'esrgan_input_dir_path': "C:\Users\pwalc\Documents\python_work_data\ESRGAN\input\",
                'esrgan_mask_dir_path': "C:\Users\pwalc\Documents\python_work_data\ESRGAN\tmp\mask\",
                'esrgan_upscaled_dir_path':"C:\Users\pwalc\Documents\python_work_data\ESRGAN\upscaled\",
                'esrgan_models_dir_path': "C:\Users\pwalc\Documents\python_work_data\ESRGAN\models\",
                'esrgan_output_dir_path': "C:\Users\pwalc\Documents\python_work_data\ESRGAN\output\",
                'esrgan_path': "C:\PythonLibraries\ESRGAN\",
                'esrgan_tmp_dir_path': "C:\Users\pwalc\Documents\python_work_data\ESRGAN\tmp\",
                'input_audio_path': "C:\Users\pwalc\Documents\python_work_data\input_files\input_audio.wav",
                'input_video_path': "C:\Users\pwalc\Documents\python_work_data\input_files\input_video.mp4",
                'output_background_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\overlay_files\output_background.mp4",
                'output_face_images_folder_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\output_face_images\",
                'silent_audio_lipsync_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\audio\silent_audio_lipsync.wav",
                'silent_audio_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\audio\silent_audio.wav",
                'wav2lip_path': "C:\PythonLibraries\Wav2Lip\",
                'wav2lip_checkpoint_wav2lip_gan_path': "C:\Users\pwalc\Documents\python_work_data\Wav2Lip\checkpoints\wav2lip_gan.pth",
                'wav2lip_checkpoint_wav2lip_path': "C:\Users\pwalc\Documents\python_work_data\Wav2Lip\checkpoints\wav2lip.pth",
            },


            'dynamic_paths': {
                'face_image_path_template': "C:\Users\pwalc\Documents\python_work_data\temp_files\output_face_images\face_{i}.png",
                'output_face_video_path_template': "C:\Users\pwalc\Documents\python_work_data\temp_files\face_video\output_face_{i}.mp4",
                'input_faces_path_template': "C:\Users\pwalc\Documents\python_work_data\temp_files\overlay_files\output_face{i}_lipsync.mp4",
            },
            'dynamic_root_paths': {
                'face_image_root_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\output_face_images\",
                'face_video_root_path': "C:\Users\pwalc\Documents\python_work_data\temp_files\face_video\",
                'overlay_files_root_path ': "C:\Users\pwalc\Documents\python_work_data\temp_files\overlay_files\"
            },
        }
        self.settings = {
            'number_of_faces': self.number_of_faces,
            'path_settings': self.path_settings,
        }
        self.default_settings = self.store_default_settings()

    def store_default_settings(self):
        return {
            'number_of_faces': self.number_of_faces,
            'path_settings': self.path_settings.copy(),
        }

    def reset_settings(self):
        self.settings = self.default_settings.copy()


class Wav2LipAppGUI(QWidget):
    def __init__(self, app):
        super().__init__()
        self.app = app
        self.init_ui()

    def init_ui(self):
        self.setWindowTitle("Wav2Lip GUI")
        self.resize(800, 600)

        main_layout = QVBoxLayout()

        # Create groupboxes
        self.create_general_settings_groupbox()
        self.create_path_settings_groupbox()
        self.create_video_audio_settings_groupbox()
        self.create_esrgan_settings_groupbox()
        self.create_blur_detection_settings_groupbox()

        # Create an accordion layout for groupboxes containing paths
        self.groupbox_toolbox = QToolBox()
        self.groupbox_toolbox.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.groupbox_toolbox.addItem(self.path_settings_groupbox, "Path Settings")
        self.groupbox_toolbox.addItem(self.video_audio_settings_groupbox, "Video & Audio Settings")
        self.groupbox_toolbox.addItem(self.esrgan_settings_groupbox, "ESRGAN Settings")
        self.groupbox_toolbox.addItem(self.blur_detection_settings_groupbox, "Blur Detection Settings")

        main_layout.addWidget(self.general_settings_groupbox)
        main_layout.addWidget(self.groupbox_toolbox)

        self.setLayout(main_layout)

    def create_path_input(self, layout, paths_dict):
        for key, value in paths_dict.items():
            path_label = QLabel(f"{key}")
            path_label.setToolTip("Enter the path for " + key)
            path_lineedit = QLineEdit()
            path_lineedit.setPlaceholderText(value)
            path_browse_button = QPushButton("Browse")
            path_browse_button.clicked.connect(partial(self.on_browse_button_click, key))
            layout.addRow(path_label, path_lineedit)
            layout.addRow("", path_browse_button)

    def create_general_settings_groupbox(self):
        self.general_settings_groupbox = QGroupBox("General Settings")
        layout = QVBoxLayout()

        # Number of faces
        faces_layout = QHBoxLayout()
        faces_label = QLabel("Number of faces:")
        faces_label.setToolTip("Set the number of faces to detect.")
        self.faces_spinbox = QSpinBox()
        self.faces_spinbox.setMinimum(1)
        self.faces_spinbox.setMaximum(10)
        self.faces_spinbox.setValue(self.app.settings['number_of_faces'])
        faces_layout.addWidget(faces_label)
        faces_layout.addWidget(self.faces_spinbox)
        layout.addLayout(faces_layout)

        # Upload MP4
        self.upload_mp4_button = QPushButton("Upload MP4")
        self.upload_mp4_button.clicked.connect(self.on_upload_mp4_button_click)
        layout.addWidget(self.upload_mp4_button)

        # Upload WAV
        self.upload_wav_button = QPushButton("Upload WAV")
        self.upload_wav_button.clicked.connect(self.on_upload_wav_button_click)
        layout.addWidget(self.upload_wav_button)

        # set all paths to directories, files and dynmic teplates
        self.set_paths_button = QPushButton("Set Paths")
        self.set_paths_button.clicked.connect(self.on_set_paths_button_click)
        layout.addWidget(self.set_paths_button)

     
        # Reset to default values
        self.reset_button = QPushButton("Reset to Default Values")
        self.reset_button.clicked.connect(self.on_reset_button_click)
        layout.addWidget(self.reset_button)

        self.general_settings_groupbox.setLayout(layout)

    def create_path_settings_groupbox(self):
        self.path_settings_groupbox = QGroupBox("Path Settings")
        layout = QFormLayout()

        self.create_path_input(layout, self.app.path_settings['base_paths'])

        self.path_settings_groupbox.setLayout(layout)

    def create_video_audio_settings_groupbox(self):
        self.video_audio_settings_groupbox = QGroupBox("Video & Audio Settings")
        layout = QFormLayout()

        self.create_path_input(layout, self.app.path_settings['static_paths'])

        self.video_audio_settings_groupbox.setLayout(layout)

    def create_esrgan_settings_groupbox(self):
        self.esrgan_settings_groupbox = QGroupBox("ESRGAN Settings")
        layout = QFormLayout()

        self.create_path_input(layout, self.app.path_settings['static_paths'])

        self.esrgan_settings_groupbox.setLayout(layout)

    def create_blur_detection_settings_groupbox(self):
        self.blur_detection_settings_groupbox = QGroupBox("Blur Detection Settings")
        layout = QFormLayout()

        self.create_path_input(layout, self.app.path_settings['static_paths'])

        self.blur_detection_settings_groupbox.setLayout(layout)

    
    def create_base_directories(self, path_settings):
        try:
            for key in path_settings['base_paths']:
                path = path_settings['base_paths'][key]
                if not os.path.exists(path):
                    os.makedirs(path)

            QMessageBox.information(self, "Success", "Base directories created successfully!")
        except Exception as e:
            QMessageBox.warning(self, "Error", str(e))

    def create_directories_and_files(self, number_of_faces, path_settings):
        try:
            # Create base directories
            self.create_base_directories(path_settings)

            # Create dynamic root paths if they don't exist
            for key in path_settings['dynamic_root_paths']:
                if not os.path.exists(path_settings['dynamic_root_paths'][key]):
                    os.makedirs(path_settings['dynamic_root_paths'][key])

            # Create dynamic paths if they don't exist
            for key in path_settings['dynamic_paths']:
                for i in range(number_of_faces):
                    path = path_settings['dynamic_paths'][key].replace('{i}', f'{i+1}')
                    directory, filename = os.path.split(path)
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                    if filename:
                        open(path, 'a').close()

            # Create static paths if they don't exist
            for key in path_settings['static_paths']:
                path = path_settings['static_paths'][key]
                directory, filename = os.path.split(path)
                if not os.path.exists(directory):
                    os.makedirs(directory)
                if filename:
                    open(path, 'a').close()

            QMessageBox.information(self, "Success", "Directories and files created successfully!")
        except Exception as e:
            QMessageBox.warning(self, "Error", str(e))

    def set_default_paths(self):
        try:
            number_of_faces = self.faces_spinbox.value()
            path_settings = self.app.path_settings

            # Create all required directories and files
            self.create_directories_and_files(number_of_faces, path_settings)

            # Set default paths and create global variables
            for main_key in path_settings:
                for subkey in path_settings[main_key]:
                    variable_name = f"{subkey}"
                    globals()[variable_name] = path_settings[main_key][subkey]

            self.app.path_settings = path_settings

            QMessageBox.information(self, "Success", "Default paths set successfully!")
        except Exception as e:
            QMessageBox.warning(self, "Error", str(e))



    def on_browse_button_click(self, key):
        try:
            options = QFileDialog.Options()
            options |= QFileDialog.ReadOnly
            file_name, _ = QFileDialog.getOpenFileName(self, "QFileDialog.getOpenFileName()", "", "All Files (*);;Text Files (*.txt)", options=options)
            if file_name:
                # Store the selected file path in the path settings dictionary
                self.app.path_settings['base_paths'][key] = file_name
                
                # Set the placeholder text of the corresponding QLineEdit to the new file path
                for i in range(self.path_settings_groupbox.layout().rowCount()):
                    item = self.path_settings_groupbox.layout().itemAt(i)
                    if isinstance(item, QHBoxLayout):
                        line_edit = item.itemAt(1).widget()
                        if line_edit.text() == self.app.path_settings['base_paths'][key]:
                            line_edit.setPlaceholderText(self.app.path_settings['base_paths'][key])
                            break
        except Exception as e:
            QMessageBox.warning(self, "Error", str(e))


    def on_upload_mp4_button_click(self):
        try:
            options = QFileDialog.Options()
            options |= QFileDialog.ReadOnly
            file_name, _ = QFileDialog.getOpenFileName(self, "QFileDialog.getOpenFileName()", "", "MP4 Files (*.mp4);;All Files (*)", options=options)
            if file_name:
                # Create the input_video_path directory if it doesn't exist
                os.makedirs(os.path.dirname(self.app.path_settings['static_paths']['input_video_path']), exist_ok=True)
                # Copy the selected MP4 file to the input_video_path
                shutil.copyfile(file_name, self.app.path_settings['static_paths']['input_video_path'])
                # Display a success message
                QMessageBox.information(self, "Success", "MP4 file uploaded successfully!")
            else:
                # Display a warning message if no file is selected
                QMessageBox.warning(self, "Warning", "No MP4 file selected.")
        except Exception as e:
            # Display an error message if an exception occurs
            QMessageBox.warning(self, "Error", str(e))



    def on_upload_wav_button_click(self):
        try:
            options = QFileDialog.Options()
            options |= QFileDialog.ReadOnly
            file_name, _ = QFileDialog.getOpenFileName(self, "QFileDialog.getOpenFileName()", "", "WAV Files (*.wav);;All Files (*)", options=options)
            if file_name:
                # Create the input_audio_path directory if it doesn't exist
                os.makedirs(os.path.dirname(self.app.path_settings['static_paths']['input_audio_path']), exist_ok=True)
                # Copy the selected WAV file to the input_audio_path
                shutil.copyfile(file_name, self.app.path_settings['static_paths']['input_audio_path'])
                # Display a success message
                QMessageBox.information(self, "Success", "WAV file uploaded successfully!")
            else:
                # Display a warning message if no file is selected
                QMessageBox.warning(self, "Warning", "No WAV file selected.")
        except Exception as e:
            # Display an error message if an exception occurs
            QMessageBox.warning(self, "Error", str(e))


    def on_set_paths_button_click(self):
        try:
            path_settings = self.set_default_paths()

            # Display a success message
            QMessageBox.information(self, "Success", "Paths set successfully!")
        except Exception as e:
            # Display an error message if an exception occurs
            QMessageBox.warning(self, "Error", str(e))

            

    def on_reset_button_click(self):
        try:
            self.faces_spinbox.setValue(self.app.settings['number_of_faces'])

            for key, value in self.app.path_settings['base_paths'].items():
                self.app.path_settings[key] = value

            for key, value in self.app.path_settings['static_paths'].items():
                self.app.path_settings[key] = value
        except Exception as e:
            QMessageBox.warning(self, "Error", str(e))


# instantiate the Wav2LipApp
app = Wav2LipApp()
# instantiate the Wav2LipAppGUI and pass the app instance
gui = Wav2LipAppGUI(app)
# show the main windoow of the Wav2LipApp()
gui.show()




In [ ]:
#@title LipSync App
import os
import shutil
import subprocess
from ipywidgets import Button, Layout, Dropdown, Output, FloatSlider, VBox, HBox, Tab, Checkbox, IntSlider, FloatProgress, Accordion, IntText, ToggleButtons
import base64
import hashlib
import random
from IPython.display import Javascript
import ipywidgets as widgets
from IPython.display import display
from functools import partial

class LipSyncApp:
    def __init__(self, get_audio_setting_callback=None):
        self.wav2lip_path = wav2lip_path
        self.wav2lip_checkpoint_wav2lip_path = wav2lip_checkpoint_wav2lip_path
        self.wav2lip_checkpoint_wav2lip_gan_path = wav2lip_checkpoint_wav2lip_gan_path
        self.get_audio_setting = get_audio_setting_callback

    @staticmethod
    def lip_sync_video(face_video_path, audio_path, output_video_path, no_smooth, resize_factor, face_det_batch_size, wav2lip_batch_size, progress_callback, box, crop, pads):
        checkpoint_path = (
            wav2lip_checkpoint_wav2lip_path
            if wav2lip_preset == "Fast"
            else wav2lip_checkpoint_wav2lip_gan_path
        )

        process = subprocess.Popen(
            [
                "python",
                f"{wav2lip_path}inference.py",
                "--checkpoint_path",
                checkpoint_path,
                "--face",
                face_video_path,
                "--audio",
                audio_path,
                "--outfile",
                output_video_path,
                "--no_smooth",
                str(no_smooth),
                "--resize_factor",
                str(resize_factor),
                "--face_det_batch_size",
                str(face_det_batch_size),
                "--wav2lip_batch_size",
                str(wav2lip_batch_size),
                "--box",
                *map(str, box),
                "--crop",
                *map(str, crop),
                "--pads",
                *map(str, pads),
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        stdout, stderr = process.communicate()

        if process.returncode != 0:
            print("Error occurred during lip-sync:")
            print(stderr.decode("utf-8"))
        else:
            print("Lip-sync process completed successfully:")
            print(stdout.decode("utf-8"))

        return output_video_path

    def lip_sync_face_video(self, index):
        audio_setting = self.get_audio_setting(index)
        video_path = f"{face_video_root_path}output_face_{index + 1}.mp4"
        
        if audio_setting == "process_audio":
            audio_path = adjusted_audio_path
        elif audio_setting == "silent_audio":
            audio_path = silent_audio_path
        else:  # "do_not_process"
            audio_path = None
        output_path = f"{overlay_files_root_path}output_face{index + 1}_lipsync.mp4"

        # Set advanced settings
        no_smooth = lip_sync_gui.advanced_settings.children[0].value
        resize_factor = lip_sync_gui.advanced_settings.children[1].value
        face_det_batch_size = lip_sync_gui.advanced_settings.children[2].value
        wav2lip_batch_size = lip_sync_gui.advanced_settings.children[3].value
        wav2lip_preset = lip_sync_gui.preset_dropdown.value
        box = lip_sync_gui.advanced_settings.children[4].value
        crop = lip_sync_gui.advanced_settings.children[5].value
        pads = lip_sync_gui.advanced_settings.children[6].value

        with self.lbl_lip_sync_progress:
            print(f"Processing Face {index + 1}...")

        if audio_path is not None:
            self.lip_sync_video(
                video_path,
                audio_path,
                output_path,
                no_smooth,
                resize_factor,
                face_det_batch_size,
                wav2lip_batch_size,
                None,
                box,
                crop,
                pads,
            )
        else:
            shutil.copy(video_path, output_path)

        return output_path



class CustomImageButton(Button):
    def __init__(self, image_path, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.layout.width = "100px"
        self.layout.height = "100px"
        self.image_path = image_path
        self.update_button_style()

    def update_button_style(self):
        with open(self.image_path, "rb") as f:
            image_data = f.read()
        base64_image_data = base64.b64encode(image_data).decode("utf-8")
        unique_identifier = hashlib.md5(
            (self.image_path + str(random.random())).encode("utf-8")
        ).hexdigest()[:10]
        button_style = f"""
             .custom-image-button-{unique_identifier} {{
                background-image: url('data:image/png;base64,{base64_image_data}');
                background-size: 100% 100%;
                background-repeat: no-repeat;
                border: none;
            }}
        """
        self.add_class(f"custom-image-button-{unique_identifier}")
        display(
            Javascript(
                f"""
            var style = document.createElement('style');
            style.innerHTML = `{button_style}`;
            document.head.appendChild(style);
        """
            )
        )
class LipSyncGUI:
    def __init__(self, num_faces=2):
        self.num_faces = num_faces
        self.selected_face = 0

        self.face_colors = ['blue', 'green', 'red', 'yellow', 'purple']
        self.face_containers = []

        self.init_face_containers()

        self.faces_tab = widgets.Tab()
        self.faces_tab.set_title(0, 'Face Sync')
        self.faces_tab.children = [widgets.VBox(self.face_containers)]

        self.init_wav2lip_settings()

        self.tab = widgets.Tab()
        self.tab.set_title(0, 'Face Sync')
        self.tab.set_title(1, 'Wav2Lip')
        self.tab.children = [self.faces_tab, self.wav2lip_settings]
        self.face_buttons = []
        for i in range(number_of_faces):
            image_path = f"face_{i+1}.png"
            face_button = CustomImageButton(
                image_path,
                layout=Layout(width="100px", height="100px"),
                description=f"Face {i+1}"
            )
            face_button.on_click(self.on_face_video_button_click)
            self.face_buttons.append(face_button)
            
    def init_face_containers(self):
        for index in range(self.num_faces):
            face_button = widgets.Button(description=f'Face {index + 1}', layout=widgets.Layout(width='auto'))
            face_button.on_click(partial(self.on_face_video_button_click, index=index))

            audio_dropdown = widgets.Dropdown(options=['Silent', 'Audio 1', 'Audio 2'], layout=widgets.Layout(width='auto'))
            audio_dropdown.observe(partial(self.on_audio_dropdown_change, index=index), names='value')

            progress_bar = widgets.FloatProgress(min=0, max=1, value=0, layout=widgets.Layout(width='auto', display='none'))
            progress_label = widgets.Label(value="0%", layout=widgets.Layout(width='auto', display='none'))

            container = widgets.VBox([face_button, audio_dropdown, progress_bar, progress_label], layout=widgets.Layout(border=f'1px solid {self.face_colors[index]}'))
            self.face_containers.append(container)

    def on_face_video_button_click(self, button, index):
        self.selected_face = index

    def on_audio_dropdown_change(self, change, index):
        selected_audio = change['new']
        self.face_containers[index].children[0].style.button_color = self.face_colors[selected_audio]

    def init_wav2lip_settings(self):
        self.wav2lip_presets = widgets.ToggleButtons(
            options=['Low Quality', 'Medium Quality', 'High Quality'],
            description='Presets:',
            disabled=False,
            button_style='',
            tooltips=['Low Quality Preset', 'Medium Quality Preset', 'High Quality Preset'],
        )

        self.wav2lip_advanced_settings = widgets.VBox([
            widgets.Label(value="Advanced Settings:"),
            # Add advanced settings widgets here
        ])

        self.wav2lip_settings = widgets.HBox([
            widgets.VBox([self.wav2lip_presets], layout=widgets.Layout(width='33%')),
            widgets.VBox([self.wav2lip_advanced_settings], layout=widgets.Layout(width='67%')),
        ])

    def display_gui(self):
        display(self.tab)

# Create an instance of the LipSyncGUI class and display the GUI
lip_sync_gui = LipSyncGUI()
lip_sync_gui.display_gui()

lip_sync_app = LipSyncApp()
lip_sync_gui = LipSyncGUI(lip_sync_app, number_of_faces)
lip_sync_gui.display()


TypeError: ignored

In [ ]:
#@title Cell 11 - overlay_faces_on_background
import cv2
import os
from tqdm import tqdm

def overlay_faces_on_background(input_background, input_faces, face_masks_list, temp_dir, output_video):
    background_capture = cv2.VideoCapture(input_background)
    fps = int(background_capture.get(cv2.CAP_PROP_FPS))
    width = int(background_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(background_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    background_capture.release()

    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    temp_background_video = os.path.join(temp_dir, "temp_background.mp4")
    
    total_frames = sum([len(face_masks) for face_masks in face_masks_list])

    progress_bar = tqdm(total=total_frames, desc="Overlaying faces", ncols=100)

    for face_video, face_masks in zip(input_faces, zip(*face_masks_list)):
        input_background_capture = cv2.VideoCapture(input_background)
        temp_background_writer = cv2.VideoWriter(temp_background_video, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

        while True:
            success, frame = input_background_capture.read()
            if not success:
                break
            temp_background_writer.write(frame)

        input_background_capture.release()
        temp_background_writer.release()

        temp_background_capture = cv2.VideoCapture(temp_background_video)
        output_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

        face_capture = cv2.VideoCapture(face_video)

        for frame_number, face_mask in enumerate(face_masks):
            success, background_frame = temp_background_capture.read()
            if not success:
                break

            success, face_frame = face_capture.read()
            if not success:
                break

            resized_face = cv2.resize(face_frame, (face_mask.shape[1], face_mask.shape[0]))
            background_frame[np.where(face_mask != 0)] = resized_face[np.where(face_mask != 0)]
            output_writer.write(background_frame)
            
            progress_bar.update(1)

        face_capture.release()
        temp_background_capture.release()
        output_writer.release()
        input_background = output_video

    os.remove(temp_background_video)
    progress_bar.close()
    print("Faces overlaid on the background.")
overlay_faces_on_background(output_temp_path, input_faces_lipsync, face_masks_list, output_temp_path, final_output_path)
print("Finished overlaying faces on the background.")



Overlaying faces:  94%|████████████████████████████████████████▍  | 494/526 [00:27<00:01, 18.23it/s]

Faces overlaid on the background.
Finished overlaying faces on the background.


In [ ]:
# Define functions
def extract_frames(video_path, output_dir):
    """Extract frames from a video file and save them as images in a directory."""
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    i = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(os.path.join(output_dir, f"frame_{i:04d}.png"), frame)
        i += 1
    cap.release()

In [ ]:
#@title Cell 12 Upscale with SuperRes-ESRGAN

image_or_directory_to_upscale = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}

#@markdown <hr size="1" color="#666">
#@markdown <small>&nbsp;</small>

#@markdown ### Advanced settings

# All models:
model = "FatalAnime" #@param ["ESRGAN_and_FatalPixels", "ESRGAN_and_PSNR", "ESRGAN", "PSNR", "FatalAnime", "FatalPixels", "NMKD", "None"]
# Selected models:
# model = "PSNR" #@param ["ESRGAN_and_PSNR", "ESRGAN", "PSNR", "FatalAnime", "FatalPixels", "deJpeg", "NMKD", "DeJPEG_only"]
## #@markdown <small>If you selected ESRGAN_and_PSNR, select interpolation ratio. 0 = ESRGAN only, 100 = PSNR only</small>
## ESRGAN_PSNR_ratio = 0 #@param {type:"slider", min:0, max:100, step:1}
#@markdown <small>When two models are selected from model menu, select ratio in which they are used. 0 = first model, 100 = second model.</small>
interpolation = 21 #@param {type:"slider", min:0, max:100, step:1}
#@markdown <small>Remove JPEG artefacts by running DeJPEG model prior to upscaling.</small>
dejpeg = True #@param {type:"boolean"}
#@markdown <small>Add G'MIC _mightly details_ effect prior to upscaling.</small>
sharpen = False #@param {type:"boolean"}
#@markdown <small>Include used model(s) and interpolation in filename. Probably helpful if you test out different settings.</small>
models_in_filename = False #@param {type:"boolean"}
#@markdown <small>Input path if you want to use models from your Drive. Otherwise models are downloaded every time you run this cell, providing their hosting servers are up and running.</small>
local_models_dir = "" #@param {type:"string"}

#----------------------------------------------------------
# #@markdown <hr color="#666">

# #@markdown <small>Timestamp output files (YYYYMMDDHHIISS).</small>
# timestamp = False #@param {type:"boolean"}
# #@markdown ### Dev options
# blur_mask = False #@param {type:"boolean"}
# blur_threshold = 39 #@param {type:"slider", min:0, max:200, step:1}
# show_progress = True #@param {type:"boolean"}
# show_result = True #@param {type:"boolean"}
timestamp = False
blur_mask = False
show_progress = False
blur_threshold = 1
show_result = False
#----------------------------------------------------------
import os
import sys

def dlfail():
    op(c.fail, 'Failed to download model, unable to proceed.')
    sys.exit()

use_local = False
if local_models_dir != '' and os.path.isdir(drive_root+local_models_dir):
    mdir = fix_path(drive_root+local_models_dir)
    if os.path.isfile(mdir+'RRDB_ESRGAN_x4.pth'):
        !cp {mdir}RRDB_ESRGAN_x4.pth /content/drive/MyDrive/colab_projects/ESRGAN/models/RRDB_ESRGAN_x4.pth
        downloaded_models.append('ESRGAN')
    if os.path.isfile(mdir+'RRDB_PSNR_x4.pth'):
        !cp {mdir}RRDB_PSNR_x4.pth  /content/drive/MyDrive/colab_projects/ESRGAN/models/RRDB_PSNR_x4.pth
        downloaded_models.append('PSNR')
    if os.path.isfile(mdir+'4x_fatal_Anime_500000_G.pth'):
        !cp {mdir}4x_fatal_Anime_500000_G.pth /content/drive/MyDrive/colab_projects/ESRGAN/models/4x_FatalAnime_500000_G.pth
        downloaded_models.append('FatalAnime')
    if os.path.isfile(mdir+'4x_FatalPixels_340000_G.pth'):
        !cp {mdir}4x_FatalPixels_340000_G.pth /content/drive/MyDrive/colab_projects/ESRGAN/models/4x_FatalPixels.pth
        downloaded_models.append('FatalPixels')
    if os.path.isfile(mdir+'1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth'):
        !cp {mdir}1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth /content/drive/MyDrive/colab_projects/ESRGAN/models/1x_DeJpeg_Fatality_PlusULTRA.pth
        downloaded_models.append('DeJpeg')
    use_local = True
else:
    dlmsg = 'Downloading required model...'
    # RRDB ESRGAN & RRDB PSNR by Xintao Wang, Ke Yu, Shixiang Wu, Jinjin Gu, Yihao Liu, Chao Dong, Yu Qiao, Chen Change Loy
    if 'ESRGAN' in model and 'ESRGAN' not in downloaded_models:
        op(c.title, dlmsg)
        !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-' -O /content/drive/MyDrive/colab_projects/ESRGAN/models/RRDB_ESRGAN_x4.pth
        if os.path.getsize('/content/drive/MyDrive/colab_projects/ESRGAN/models/RRDB_ESRGAN_x4.pth') > 2000:
            downloaded_models.append('ESRGAN')
        else:
            dlfail()
    if 'PSNR' in model and 'PSNR' not in downloaded_models:
        op(c.title, dlmsg)
        !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr' -O /content/drive/MyDrive/colab_projects/ESRGAN/models/RRDB_PSNR_x4.pth
        if os.path.getsize('/content/drive/MyDrive/colab
    # fatal_anime by twittman
    if 'FatalAnime' in model and 'FatalAnime' not in downloaded_models:
      op(c.title, dlmsg)
      !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download" -O /content/drive/MyDrive/colab_projects/ESRGAN/models/4x_FatalAnime_500000_G.pth
      if os.path.getsize('/content/drive/MyDrive/colab_projects/ESRGAN/models/4x_FatalAnime_500000_G.pth') > 2000:
        downloaded_models.append('FatalAnime')
      else:
        dlfail()
    # fatal_pixels twittman
    if 'FatalPixels' in model and 'FatalPixels' not in downloaded_models:
      op(c.title, dlmsg)
      !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/mDGmi7NgdyyQRXL/download?path=%2F&files=4x_FatalPixels_340000_G.pth&downloadStartSecret=r4q3aw60ijm" -O /content/drive/MyDrive/colab_projects/ESRGAN/models/4x_FatalPixels.pth
      if os.path.getsize('/content/drive/MyDrive/colab_projects/ESRGAN/models/4x_FatalPixels.pth') > 2000:
        downloaded_models.append('FatalPixels')
      else:
        dlfail()
    # deJpeg_Fatality_PlusULTRA by twittman
    if ('DeJpeg' in model or dejpeg is True) and 'DeJpeg' not in downloaded_models:
      op(c.title, dlmsg)
      !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download" -O /content/drive/MyDrive/colab_projects/ESRGAN/models/1x_DeJpeg_Fatality_PlusULTRA.pth
      if os.path.getsize('/content/drive/MyDrive/colab_projects/ESRGAN/models/1x_DeJpeg_Fatality_PlusULTRA.pth') > 2000:
        downloaded_models.append('DeJpeg')
      else:
        dlfail()
    # 4x_NMKD-YandereNeo-Lite_320k by nmkd
    if 'NMKD' in model and 'NMKD' not in downloaded_models:
      op(c.title, dlmsg)
      !gdown --id 14lA-Ks5quxheNyVeXRvzeoSAOm6ISDHn -O "/content/drive/MyDrive/colab_projects/ESRGAN/models/4x_NMKD-YandereNeo-Lite_320k.pth"
      if os.path.getsize('/content/drive/MyDrive/colab_projects/ESRGAN/models/4x_NMKD-YandereNeo-Lite_320k.pth') > 2000:
        downloaded_models.append('NMKD')
      else:
        dlfail()
    # output.clear()

    #----------------------------------------------------------

sharpen_before = True
sharpen_after = False

show_state = show_progress

if os.listdir(dir_input):
    clean_dirs([dir_input, dir_output, dir_mask, dir_dejpeg, dir_upscaled])

image_or_directory_to_upscale = '/content/drive/MyDrive/' + image_or_directory_to_upscale

is_dir = False
if os.path.isfile(image_or_directory_to_upscale):
    input_images = [image_or_directory_to_upscale]
elif os.path.isdir(image_or_directory_to_upscale):
    input_images = list_images(image_or_directory_to_upscale)
    input_images.sort()
    is_dir = True
else:
    op(c.fail, 'Images not found!')
    print('This path does not point to a file or directory: '+image_or_directory_to_upscale.replace(drive_root, ''))
    sys.exit()

if output_dir == '' or output_dir == 'same':
    output_dir = path_dir(input_images[0])
else:
    output_dir = '/content/drive/MyDrive/' + output_dir
    if os.path.isdir(output_dir):
        output_dir_images = list_images(output_dir)
    else:
        create_dirs([output_dir])

uniq_id = gen_id()

model_1_part = 100 - interpolation
model_2_part = interpolation

# print('esrgan', model_1_part, 'fatalpx', model_2_part)

if model == 'ESRGAN_and_FatalPixels':
    # 25 75
    model_file = 'RRDB_ESRGAN_x4.pth@'+str(model_1_part)+'|4x_FatalPixels.pth@'+str(model_2_part)
if model == 'ESRGAN_and_PSNR':
    # 70 30
    model_file = 'RRDB_ESRGAN_x4.pth@'+str(model_1_part)+'|RRDB_PSNR_x4.pth@'+str(model_2_part)
if model == 'ESRGAN':
    model_file = 'RRDB_ESRGAN_x4.pth'
if model == 'PSNR':
    model_file = 'RRDB_PSNR_x4.pth'
if model == 'FatalAnime':
    model_file = '4x_FatalAnime_500000_G.pth'
if model == 'FatalPixels':
    model_file = '4x_FatalPixels.pth'
if model == 'DeJpeg':
    model_file = '1x_DeJpeg_Fatality_PlusULTRA.pth'
if model == 'NMKD':
    model_file = 'NMKD-YandereNeo_lite_320k_x4.pth'

op(c.title, 'Run ID:', uniq_id)
op(c.title, 'Images:', len(input_images))
if use_local is True:
    print('Using models from local dir', local_models_dir)
print('\n')

exists = []
broken = []
skipped = []

for input_image in input_images:
    clean_dirs([dir_input, dir_output, dir_mask, dir_dejpeg, dir_upscaled])
    if models_in_filename:
        final_file = output_dir+basename(input_image)+'_superres_'+uniq_id+'_'+model+'_'+str(model_1_part)+'_'+str(model_2_part)+'.png'
    else:
        final_file = output_dir+basename(input_image)+'_superres_'+uniq_id+'.png'

    existing = glob(output_dir+basename(input_image)+'_superres_*.png')
    incoming = output_dir+basename(input_image)
    file_exists = False
    input_image_d = input_image.replace(drive_root, '')

    for existing_item in existing:
        if incoming in existing_item:
            file_exists = True

    if file_exists:
        op(c.warn, 'Upscaled image seems to already exist. Skipping', input_image_d)
        exists.append(input_image_d)
        skipped.append(input_image_d)
    elif os.path.getsize(input_image) is 0:
        op(c.fail, 'Image is broken. Skipping', input_image_d)
        broken.append(input_image_d)
        skipped.append(input_image_d)
    else:
        state = input_image
        !cp "{input_image}" "{dir_input}"

        op(c.title, 'Processing image:', path_leaf(input_image))

        if dejpeg:
            op(c.okb, 'DeJpeg...')
            !python upscale.py 1x_DeJpeg_Fatality_PlusULTRA.pth --input "{dir_input}" --output "{dir_dejpeg}"
            state = dir_dejpeg+os.listdir(dir_dejpeg)[0]
            if show_state: Image(state)
            op(c.ok, 'Done.')

## Sharpen
if sharpen and sharpen_before:
    op(c.okb, 'Sharpen...')
    tmp_file = dir_tmp+basename(input_image)+'_sharpened.png'
    gmic.run("input "+state+" fx_mighty_details 35,1,2,1,11,0 output "+tmp_file)
    # !cp "{tmp_file}" "{dir_input}"
    # state = input_file
    # state = dir_input+os.listdir(dir_input)[0]
    state = tmp_file
    if show_state: 
        Image(state)
    op(c.ok, 'Done.\n')

# print(input_image)
# print(state)

if state is not input_image:
    !cp "{state}" "{dir_input}"

## Neural upscale
if model is not 'None' and model is not 'DeJPEG_only':
    op(c.okb, 'Upscale...')
    !python upscale.py "{model_file}" --input "{dir_input}" --output "{dir_upscaled}"
    # state = state.replace('input/', 'output/')
    state = dir_upscaled+os.listdir(dir_upscaled)[0]
    if show_state: 
        Image(state)
    op(c.ok, 'Done.')

## Sharpen
if sharpen and sharpen_after:
    op(c.okb, 'Sharpen...')
    tmp_file = dir_tmp+basename(input_image)+'_sharpened.png'
    gmic.run("input "+state+" fx_mighty_details 35,1,2,1,11,0 output "+tmp_file)
    state = tmp_file
    if show_state: 
        Image(state)
    op(c.ok, 'Done.\n')

!cp "{state}" "{final_file}"
if os.path.isfile(final_file):
    op(c.ok, 'Superres file saved as:', final_file.replace(drive_root, '')+'\n')
if show_result: 
    Image(final_file)

## Blur map
##
## To handle blurred areas differently. Work in progress.
##
if blur_mask:
    # blur_input_img = state
    blur_input_img = input_image
    blurmapSS = dir_tmp+'blur_mask_ss'
    blurmapSL = dir_tmp+'blur_mask_sl'
    blurmapBL = dir_tmp+'blur_mask_bl.png'
    %cd /content/BlurDetection2

    op(c.title, 'Generate blur map...')
    !python process.py -i "{blur_input_img}" -s {blurmapSS} -t "{blur_threshold}" -f
    state = blurmapSS+'_blur_map.png'
    if show_state: 
        Image(state)
    op(c.ok, 'Done.\n')

    input_size = cv2.imread(state)
    width = int(input_size.shape[1])
    height = int(input_size.shape[0])
    dim = (width, height)
    temp_image = cv2.imread(state)
    upscaled = cv2.resize(temp_image, dim, interpolation = cv2.INTER_AREA)

    state = blurmapSL+'_blur_map.png'
    if show_state: 
        Image(state)
    cv2.imwrite(state, upscaled)
    ba = str(width/200)
    gmic.run("-input "+blurmapSL+'_blur_map.png'+" -blur_linear "+ba+","+ba+",0 -glow 5 -adjust_colors 0,0,10,0,0 -adjust_colors 10,50,0,0,0 -output "+blurmapBL)
 if show_state: 
    Image(blurmapBL)
op(c.ok, 'Done.\n')

!cp /content/tmp/blur_mask_bl.png /content/drive/MyDrive/ai/st3/test/
!cp /content/tmp/blur_mask_sl_blur_map.png /content/drive/MyDrive/ai/st3/test/
!cp /content/tmp/blur_mask_ss_blur_map.png /content/drive/MyDrive/ai/st3/test/
!cp /content/tmp/blur_mask_ss_pretty_blur_map.png /content/drive/MyDrive/ai/st3/test/

if len(exists) > 0:
    op(c.warn, '\nImages that were not upscaled due to being previously upscaled and existing already:')
    for image in exists:
        print('-', image)

if len(broken) > 0:
    op(c.fail, '\nImages that were not upscaled due to being broken:')
    for image in broken:
        print('-', image)

op(c.title, '\nFIN.')


SyntaxError: ignored

In [ ]:
#@title Cell 13 - Create final mp4 with Audio
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeVideoClip

# Load the silent video
video_clip = VideoFileClip("/content/work_data/output_files/final_output.mp4")

# Load the audio file
audio_clip = AudioFileClip("/content/work_data/temp_files/adjusted_audio.wav")

# Combine the video and audio clips
final_clip = CompositeVideoClip([video_clip.set_audio(audio_clip)])

# Write the final clip to a new file
final_clip.write_videofile("/content/work_data/output_files/final_out.mp4")

In [ ]:
#@title Cell 14 CLEAN-UP
print("CLEAN-UP started...\n")

def clean_up(file_paths):
    for file_path in file_paths:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Removed: {file_path}")
        else:
            print(f"File not found: {file_path}")

confirmation = input("Do you want to clean up temporary files? (y/n): ")
if confirmation.lower() == 'y':
    files_to_remove = [
        output_background_path,
        *face_image_path_template,
        output_audio_adjusted,
        output_video_adjusted,
        silent_audio_path,
        silent_audio_lipsync_path,
        temp_output_video,
        input_video_path,
        input_audio,
        input_video,
        audio_file_path
    ]
    clean_up(files_to_remove)
    print("--- clean Up done!   ---")
else:
    print("\n---   Clean Up canceled   ---")


CLEAN-UP started...

Do you want to clean up temporary files? (y/n): y
File not found: /content/work_data/temp_files/output_background.mp4
File not found: /content/work_data/temp_files/output_face1.mp4
File not found: /content/work_data/temp_files/output_face2.mp4
File not found: /content/work_data/temp_files/adjusted_audio.wav
File not found: /content/work_data/temp_files/adjusted_video.mp4
File not found: /content/work_data/temp_files/silent_audio.wav
File not found: /content/work_data/temp_files/silent_audio_lipsync.wav
File not found: /content/work_data/temp_files/temp_output_video.mp4
Removed: /content/work_data/input_files/input_video.mp4
Removed: /content/work_data/input_files/input_audio.wav
File not found: /content/work_data/input_files/input_video.mp4
File not found: /content/work_data/input_files/input_audio.wav
Deleted:  {'/content/work_data/temp_files/output_background.mp4'}
Deleted:  {'/content/work_data/temp_files/output_face1.mp4'}
Deleted:  {'/content/work_data/temp_fi